In [23]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sc
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

def dataset(dataFile, labelFile):
    dataFile = "./communication/" + dataFile
    labelFile = "./communication/" + labelFile
    my_data = sc.loadmat(dataFile)
    my_labels = sc.loadmat(labelFile)
    my_data = my_data['Y']
    X = my_labels['L_S_x']
    myOrig = table_data(my_data, my_labels['L_Constellations'][0], X)
    myTable = assign_labels(myOrig)
    return myTable

def table_data(my_data, cons, label):
    block = my_data.shape[1]
    my_data_size = my_data.shape[0] * block
    my_data_div = my_data.T.reshape(my_data_size,)
    cons_array = np.array([[cons[i]]*my_data.shape[0] for i in range(0,block)]).reshape(my_data_size,)
    block_array = np.array([([i+1]*my_data.shape[0])for i in range(0, block)]).reshape(my_data_size,)
    label_array = label.T.reshape(my_data_size,)
    test_pd = pd.DataFrame({'real':my_data_div.real,'imag':my_data_div.imag,
            'cons':cons_array, 'block':block_array,
            'label':label_array})
    return test_pd

def assign_labels(myTable):
    myTest = myTable.copy()
    myTest.loc[myTest.cons==2, 'label'] = myTest.loc[myTest.cons==2, 'label']+4
    myTest.label = myTest.label-1
    return myTest


from numpy.random import default_rng
def training_set(myTable):
    block = myTable.shape[0]
    rng = default_rng()
    sample_size = int(0.8 * block)
    numbers = rng.choice(range(1, block + 1), size=sample_size, replace=False)
    training_dataset = myTable[myTable.block.isin(numbers)]
    return training_dataset

def test_set(myTable, training_dataset):
    remaining = myTable.drop(training_dataset.index)
    return remaining



def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(50, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(20)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(0.01),
                 metrics=['accuracy'])
    return model

def get_total_loss(predict_label, true_label):
    i = 0
    for j in range(len(predict_label)):
        prediction = np.argmax(predict_label[j])
        if prediction != true_label[j]:
            i = i + 1
    return i/len(predict_label)

datas = "INR30data"
labels = "INR30labels"


def get_training(myTable, epochs, files):
    train_dataset = training_set(myTable)
    test_dataset = test_set(myTable, train_dataset)
    train_features = train_dataset.copy()
    test_features = test_dataset.copy()
    train_labels = pd.DataFrame([train_features.pop('label')]).T
    test_labels = pd.DataFrame([test_features.pop('label')]).T
    normalizer = preprocessing.Normalization()
    normalizer.adapt(np.array(train_features))
    dnn_real_model = build_and_compile_model(normalizer)
    history = dnn_real_model.fit(
        train_features, train_labels,
        validation_split=0.2,
        epochs=epochs)
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    test_results = {}
    test_results['signal'] = dnn_real_model.evaluate(
    test_features,
    test_labels, verbose=0)
    probability_model = tf.keras.Sequential([dnn_real_model, 
                                             tf.keras.layers.Softmax()])
    predictions = probability_model.predict(test_features)
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist.tail()
    loss = get_total_loss(predictions, test_labels.to_numpy())
    hist = hist.append(test_results, ignore_index=True)
    print(test_results)
    return dnn_real_model


def prediction(model, predictionDataSet, predictionLabels):
    test_dataset = dataset(predictionDataSet, predictionLabels)
    test_features = test_dataset.copy()
    test_labels = pd.DataFrame([test_features.pop('label')]).T
    predictions = model.predict(test_features)
    test_results = {}
    test_results['signal'] = model.evaluate(
    test_features,
    test_labels, verbose=0)
    return test_results
    

In [18]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [Ireal]')
    plt.legend()
    plt.grid(True)

In [19]:
results = dnn_real_model.evaluate(test_features, test_labels)

NameError: name 'dnn_real_model' is not defined

In [20]:
myTable = dataset("INR30data", "INR30labels")

In [21]:
datas = "INR30data"
labels = "INR30labels"

In [24]:
model = get_training(myTable, 10, "new_test")

Epoch 1/10
1015/1015 [==============================] - 1s 729us/step - loss: 0.9204 - accuracy: 0.6714 - val_loss: 0.6973 - val_accuracy: 0.7064
Epoch 2/10
1015/1015 [==============================] - 1s 625us/step - loss: 0.5775 - accuracy: 0.7428 - val_loss: 0.4945 - val_accuracy: 0.7695
Epoch 3/10
1015/1015 [==============================] - 1s 627us/step - loss: 0.4512 - accuracy: 0.7856 - val_loss: 0.3872 - val_accuracy: 0.8234
Epoch 4/10
1015/1015 [==============================] - 1s 645us/step - loss: 0.3761 - accuracy: 0.8164 - val_loss: 0.4031 - val_accuracy: 0.8143
Epoch 5/10
1015/1015 [==============================] - 1s 629us/step - loss: 0.3437 - accuracy: 0.8296 - val_loss: 0.3945 - val_accuracy: 0.8143
Epoch 6/10
1015/1015 [==============================] - 1s 629us/step - loss: 0.3253 - accuracy: 0.8378 - val_loss: 0.3913 - val_accuracy: 0.8252
Epoch 7/10
1015/1015 [==============================] - 1s 644us/step - loss: 0.3115 - accuracy: 0.8416 - val_loss: 0.2975 -

In [ ]:
anotherTable = dataset("my_data", "my_labels")

In [73]:
prediction(model, "INR30data", "INR30labels")

{'signal': [0.2934909462928772, 0.8456000089645386]}

In [70]:
new_test = pd.concat([myTable,anotherTable], ignore_index=True)

In [75]:
(a+"b") = 1

SyntaxError: can't assign to operator (<ipython-input-75-325436e253a2>, line 1)

In [14]:
test_results = {'signal': [0.2934909462928772, 0.8456000089645386]}

In [89]:
print("signal", test_results['signal'])

signal [0.2934909462928772, 0.8456000089645386]


In [15]:
pd.DataFrame(test_results)

,signal
0,0.293491
1,0.845600
